Data exploration
PEARSON --> continous
cramer/phi-k --> non nominal

text

In [ ]:
text
'Country_Name','State_Island_Province_Name','City_Town_Name','Site_Name','Ecoregion_Name'

unsupported

In [ ]:
'Distance_to_Shore','Turbidity','Percent_Bleaching'

cat

In [ ]:
'Exposure','Substrate_Name'
'Data_Source','Ocean_Name','Realm_Name','Exposure',
'Windspeed'
#remove
'Bleaching_Level', 'SSTA_Mean'


num

In [ ]:
'Latitude_Degrees	','Longitude_Degrees	', 
'Cyclone_Frequency','Date_Day','Date_Month','Date_Year'

after convert

In [ ]:
num_fea = ['Cyclone_Frequency','Date_Day', 'Date_Month', 'Date_Year',
'Percent_Cover','Depth_m','ClimSST',
'Temperature_Kelvin','Temperature_Mean','Temperature_Minimum','Temperature_Maximum',
'Temperature_Kelvin_Standard_Deviation',
'Windspeed',
'SSTA','SSTA_Standard_Deviation', 'SSTA_Minimum','SSTA_Maximum','SSTA_Frequency',
'SSTA_Frequency_Standard_Deviation','SSTA_FrequencyMax','SSTA_FrequencyMean',
'SSTA_DHW','SSTA_DHW_Standard_Deviation','SSTA_DHWMax','SSTA_DHWMean',
'TSA','TSA_Standard_Deviation','TSA_Minimum','TSA_Maximum','TSA_Mean',
'TSA_Frequency','TSA_Frequency_Standard_Deviation','TSA_FrequencyMax','TSA_FrequencyMean',
'TSA_DHW','TSA_DHW_Standard_Deviation','TSA_DHWMax','TSA_DHWMean']

In [ ]:
simple impute
'Distance_to_Shore ','Turbidity','Percent_Cover','ClimSST','Deph_m'

convert to num

In [ ]:
'Percent_Cover','Depth_m','ClimSST','Temperature_Kelvin','Temperature_Mean','Temperature_Minimum','Temperature_Maximum'
'Temperature_Kelvin_Standard_Deviation'
'Windspeed'
'SSTA','SSTA_Standard_Deviation','SSTA_Mean','SSTA_Minimum','SSTA_Maximum','SSTA_Frequency'
'SSTA_Frequency_Standard_Deviation','SSTA_FrequencyMax','SSTA_FrequencyMean'
'SSTA_DHW','SSTA_DHW_Standard_Deviation','SSTA_DHWMax','SSTA_DHWMean',
'TSA','TSA_Standard_Deviation','TSA_Minimum','TSA_Maximum','TSA_Mean',
'TSA_Frequency','TSA_Frequency_Standard_Deviation','TSA_FrequencyMax','TSA_FrequencyMean'
'TSA_DHW','TSA_DHW_Standard_Deviation','TSA_DHWMax','TSA_DHWMean'




In [ ]:
condition = dataset['Percent_Cover'] != 'nd'
mean = dataset[condition]['Percent_Cover'].mean()
dataset['Percent_Cover'] = dataset['Percent_Cover'].replace('nd', mean)

In [ ]:
df['city'].unique()

onehotencoder

In [ ]:
class MyFeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_TOTAL_NO_OF_ROOMS = True): # MUST NO *args or **kargs
        self.add_TOTAL_NO_OF_ROOMS = add_TOTAL_NO_OF_ROOMS
    def fit(self, feature_values, labels = None):
        return self  # nothing to do here
    def transform(self, feature_values, labels = None):
        if self.add_TOTAL_NO_OF_ROOMS:        
            NO_OF_ROOMS_id, NO_OF_TOILETS_id = 1, 2 # column indices in num_feat_names. can't use column names b/c the transformer SimpleImputer removed them
            # NOTE: a transformer in a pipeline ALWAYS return dataframe.values (ie., NO header and row index)
            
            TOTAL_NO_OF_ROOMS = feature_values[:, NO_OF_ROOMS_id] + feature_values[:, NO_OF_TOILETS_id]
            feature_values = np.c_[feature_values, TOTAL_NO_OF_ROOMS] #concatenate np arrays
        return feature_values



In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
ohetransform = ohe.fit_transform(dataset[['Exposure','Substrate_Name', 'Data_Source','Ocean_Name','Realm_Name']])
ohetransform    

fuse into dataset

In [ ]:
dataset = pd.concat([dataset,ohetransform], axis=1).drop(columns=cat_fea)

We will handle missing data for these following numeric features:

In [ ]:
num_cols = ['Latitude_Degrees', 'Longitude_Degrees', 'Distance_to_Shore',
       'Turbidity', 'Cyclone_Frequency', 'Date_Day', 'Date_Month', 'Date_Year',
       'Percent_Cover', 'ClimSST', 'Temperature_Kelvin',
       'Temperature_Mean', 'Temperature_Minimum', 'Temperature_Maximum',
       'Temperature_Kelvin_Standard_Deviation', 'Windspeed', 'SSTA',
       'SSTA_Standard_Deviation', 'SSTA_Minimum', 'SSTA_Maximum',
       'SSTA_Frequency', 'SSTA_Frequency_Standard_Deviation',
       'SSTA_FrequencyMax', 'SSTA_FrequencyMean', 'SSTA_DHW',
       'SSTA_DHW_Standard_Deviation', 'SSTA_DHWMax', 'SSTA_DHWMean', 'TSA',
       'TSA_Standard_Deviation', 'TSA_Minimum', 'TSA_Maximum', 'TSA_Mean',
       'TSA_Frequency', 'TSA_Frequency_Standard_Deviation', 'TSA_FrequencyMax',
       'TSA_FrequencyMean', 'TSA_DHW', 'TSA_DHW_Standard_Deviation',
       'TSA_DHWMax', 'TSA_DHWMean', 'Depth_m']

In [ ]:
mean = np.mean(train_set_labels)
print(mean)
train_set_labels = np.where(np.nan, train_set_labels, mean)

# save the figure to a Bytesting

canvas = agg.FigureCanvasAgg(fig)
canvas.draw()
renderer = canvas.get_renderer()
raw_data = renderer.tostring_rgb()
width, height = fig.get_size_inches()*fig.dpi

#convert Bytestring to Image
Image = PIL.Image.frombytes("RGB", (width, height), raw_data)


IQR

In [ ]:
# outlier_IQR = outliers_IQR(dataset, dataset.select_dtypes("number"))

In [ ]:
#handle outliers with z-score


outlier = []
def outliers_Z(data):
    threshold = 3
    count = 0

    for i in data:
        mean = np.mean(data[i]) 
        std = np.std(data[i])
        for j in data[i]:
         
            z = (j-mean)/std 
            if np.abs(z) > threshold: 
                # notice outliers and remove it
                data = data[data[i] != j]
                count+=1
    return data


In [ ]:

dataset_after_outliers = outliers_Z(dataset.select_dtypes("number"))
dataset1 = dataset.update(dataset_after_outliers)
dataset1.head()

In [ ]:
dataset1 = dataset.copy(deep=True)
dataset1.shape

In [ ]:
SimImp = ['Distance_to_Shore','Turbidity','Percent_Cover','ClimSST','Deph_m']

In [ ]:
num_pipeline = Pipeline([
    ('selector', ColumnSelector(SimImp)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="median", copy=True)) # copy=False: imputation will be done in-place 
    ])  
      ('imputer', KNNImputer(missing_values=np.nan, n_neighbors=2)), # copy=False: imputation will be done in-place 


In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.feature_names = feature_names
    def fit(self, dataframe, labels=None):
        return self
    def transform(self, dataframe):
        return dataframe[self.feature_names].values     

In [ ]:
cat_pipeline = Pipeline([
    ('selector', ColumnSelector(cat_fea)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="constant", fill_value = "NO INFO", copy=True)), # complete missing values. copy=False: imputation will be done in-place 
    ('cat_encoder', OneHotEncoder()) # convert categorical data into one-hot vectors
    ])
trans_fea = cat_pipeline.fit_transform(train_set)

In [ ]:
num_pipeline = Pipeline([
    ('selector', ColumnSelector(num_cols)),
    ('imputer', SimpleImputer(missing_values = np.nan, strategy="median", copy=False)) # copy=False: imputation will be done in-place 
    ])  

In [ ]:
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline) ])  

In [ ]:
processed_train_set_val = full_pipeline.fit_transform(train_set)
print(processed_train_set_val.shape)

print('\n____________ Processed feature values ____________')
print(processed_train_set_val[[0, 1, 2],:].toarray())

print('We have %d numeric feature + cols of onehotvector for categorical features.' %(len(num_for_missing)))